### 1. 모듈

In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
import time
import pandas as pd
import re

### 2. url 접속

In [9]:
driver = webdriver.Chrome('C:/Users/NK/Desktop/programming/python/chromedriver.exe')
url = 'http://likms.assembly.go.kr/bill/main.do'
driver.get(url)

In [10]:
search = driver.find_element_by_css_selector('li.gnbL2')
search.click()
time.sleep(2)

driver.find_element_by_css_selector('#AGE_AREA > select:nth-child(2) > option:nth-child(2)').click()

search = driver.find_element_by_css_selector('div.mt20.alignC > button.btnd')
search .click()
time.sleep(2)

more_bill = driver.find_element_by_css_selector('#pageSizeOption > option:nth-child(4)').click()

### 3. 크롤링

In [11]:
num_list = []
proposer_list = []

In [12]:
n_times = 0

## 원하는 시작페이지
n_iter = 0

driver.find_element_by_css_selector('#pageListViewArea > a:nth-child(11)').click()
while n_times <= n_iter - 2:
    driver.find_element_by_css_selector('#pageListViewArea > a:nth-child(13)').click()
    n_times +=1

In [ ]:
if n_iter == 0:
    start = 1; end = 11
else:
    start = 3; end = 13
    
## 10페이지가 끝나면 다음 10페이지를 불러옵니다.
while n_iter <= 2:

    ## start번째 페이지에 접속합니다.
    for k in tqdm_notebook(range(start,end)):
        driver.find_element_by_css_selector(f'#pageListViewArea > a:nth-child({k})').click()
        ## start번째 페이지의 i번째 의안에 접근합니다.
        for i in range(1,101):
            ## i 번쨰 의안에 접근합니다.
            bill = driver.find_element_by_css_selector(f'body > div > div.contentWrap > div.subContents > div > div.tableCol01 > table > tbody > tr:nth-child({i})')

            num = bill.find_element_by_css_selector('td:nth-child(1)').text
            num_list.append(num)

            ## start번째 페이지의 i번째 의안에 접속합니다.
            bill = driver.find_element_by_css_selector(f'body > div > div.contentWrap > div.subContents > div > div.tableCol01 > table > tbody > tr:nth-child({i}) > td.alignL > div.pl25 > a')
            bill.click()

            ## 제안자를 담습니다.
            member_list = []    
            try:
                proposer = driver.find_element_by_css_selector('body > div > div.contentWrap > div.subContents > div > div.contIn > div.tableCol01 > table > tbody > tr > td:nth-child(3) > a > img')
                proposer.click()

                # 새로운 창으로 초점을 바꿉니다.
                driver.switch_to_window(driver.window_handles[1])
                driver.get_window_position(driver.window_handles[1])

                member_num = len(driver.find_elements_by_tag_name('a'))
                member_end = (member_num//3) + member_num + 2

                for j in range(2, member_end):
                    try:
                        member = driver.find_element_by_css_selector(f'#periodDiv > div.layerInScroll.coaTxtScroll > div > a:nth-child({j})').text
                        member_list.append(member)
                    except:
                        continue
                        
                driver.close()
                proposer_list.append(member_list)

                # 원래 창으로 초점을 바꿉니다.
                driver.switch_to_window(driver.window_handles[0])
                driver.get_window_position(driver.window_handles[0])

            except:
                proposer_list.append([])
    
            driver.back()
            driver.execute_script("window.scrollTo(0, window.scrollY + 50)")

    driver.find_element_by_css_selector(f'#pageListViewArea > a:nth-child({10+start})').click()
    if n_iter == 0:
        start = 1; end = 11
    else:
        start = 3; end = 13
    n_iter += 1

C:\Users\NK\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


C:\Users\NK\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: DeprecationWarning: use driver.switch_to.window instead
C:\Users\NK\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: DeprecationWarning: use driver.switch_to.window instead


### 4. 데이터프레임

In [ ]:
result = [num_list, proposer_list]
result_df = pd.DataFrame(result).T
result_df.columns = ['의안번호', '제안자']
result_df

In [ ]:
result3.to_csv('20대 국회 의안정보(5).csv', encoding='utf-8-sig', index=False)